<a href="https://colab.research.google.com/github/ankit-rathi/Data-Science-with-Python/blob/master/Stock_Analysis/Nifty100_Upside_Analysis_Forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
#!pip install talib-binary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np

mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Data-Science-with-Python/master/Stock_Analysis/data/Nifty100Scrips.csv')

mypf = mypf[mypf['InScope'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100

In [3]:
mypf = mypf[mypf.Forecast.notnull()]
stock_ids = mypf[mypf.Forecast.notnull()]['Symbol'].unique()

In [4]:
stock_ids.sort()
stock_ids

array(['ASIANPAINT', 'AXISBANK', 'BAJAJFINSV', 'BAJFINANCE', 'BANKBARODA',
       'BERGEPAINT', 'BHARTIARTL', 'BRITANNIA', 'CANBK', 'CIPLA',
       'COLPAL', 'DABUR', 'DEEPAKNTR', 'DIVISLAB', 'DMART', 'DRREDDY',
       'EICHERMOT', 'FEDERALBNK', 'GODREJCP', 'GODREJIND', 'HAVELLS',
       'HCLTECH', 'HDFC', 'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO',
       'HINDUNILVR', 'IBREALEST', 'ICICIBANK', 'ICICIPRULI', 'INDIANB',
       'INDIGO', 'INDUSINDBK', 'INFY', 'IRCTC', 'ITC', 'JUBLFOOD',
       'KOTAKBANK', 'LT', 'LTIM', 'M&M', 'MARUTI', 'MCDOWELL-N',
       'MOTILALOFS', 'MUTHOOTFIN', 'NATIONALUM', 'NAUKRI', 'PIDILITIND',
       'RELIANCE', 'SAIL', 'SBILIFE', 'SBIN', 'SUNPHARMA', 'TATACONSUM',
       'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN',
       'ULTRACEMCO', 'WIPRO', 'ZEEL'], dtype=object)

In [5]:
mypf['InPortfolio'].value_counts()

0    54
1     9
Name: InPortfolio, dtype: int64

In [6]:
# import necessary libraries 

%matplotlib inline
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from tabulate import tabulate
import warnings
warnings.filterwarnings('ignore')
import pandas_datareader.data as web
import yfinance as yfin
#import talib 
yfin.pdr_override()

In [7]:
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
    #end_date = '2020-01-01'
    end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    short_window = 20 
    mid_window = 50
    long_window = 200
    moving_avg = 'EMA'


    # import the closing price data of the stock for the aforementioned period of time in Pandas dataframe
    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-'))) 
    stock_df = web.DataReader(stock_symbol,  start = start, end = end)['Close']
    stock_df = pd.DataFrame(stock_df) # convert Series object to dataframe 
    stock_df.columns = {'Close Price'} # assign new colun name
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows 

    # column names for long and short moving average columns
    short_window_col = str(short_window) + '_' + moving_avg
    mid_window_col = str(mid_window) + '_' + moving_avg
    long_window_col = str(long_window) + '_' + moving_avg  

    # Create short exponential moving average column
    stock_df[short_window_col] = round(stock_df['Close Price'].ewm(span = short_window, adjust = False).mean(),0)

    stock_df[mid_window_col] = round(stock_df['Close Price'].ewm(span = mid_window, adjust = False).mean(),0)     

    #stock_df['RSI'] = round(talib.RSI(stock_df['Close Price'], timeperiod=14),0)

    # Create a long exponential moving average column
    stock_df[long_window_col] = round(stock_df['Close Price'].ewm(span = long_window, adjust = False).mean(),0)
    stock_df['Stock'] = stock_symbol
    stock_df['Dev%_200'] = round((stock_df['Close Price'] - stock_df['200_EMA'])*100/stock_df['200_EMA'],2)
    stock_df['200_EMA_Lag10'] = stock_df['200_EMA'].shift(10)
    stock_df['Slope%_200'] = round((stock_df['200_EMA'] - stock_df['200_EMA_Lag10'])*100/stock_df['200_EMA'],2)
    stock_df['50_EMA_Lag5'] = stock_df['50_EMA'].shift(5)
    stock_df['Slope%_50'] = round((stock_df['50_EMA'] - stock_df['50_EMA_Lag5'])*100/stock_df['50_EMA'],2)
    stock_df['Max'] = round(max(stock_df['Close Price']),0)
    stock_df['Min'] = round(min(stock_df['Close Price']),0)    
    #stock_df['Dev%_Max'] = round((stock_df['Close Price'] - stock_df['Max'])*100/stock_df['Max'],2)
    #stock_df['Dev%_Min'] = round((stock_df['Close Price'] - stock_df['Min'])*100/stock_df['Min'],2)
    stock_df['Premium%'] = round((stock_df['Close Price']-stock_df['Min'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Discount%'] = round((stock_df['Max']-stock_df['Close Price'])*100/(stock_df['Max']-stock_df['Min']),2)
    stock_df['Close Price'] = round(stock_df['Close Price'],0) 
    stock_df.drop(['20_EMA','50_EMA','200_EMA','200_EMA_Lag10','50_EMA_Lag5'], axis=1, inplace=True)
    
    
    return stock_df

df_prec_dev = pd.DataFrame()

#tmp = stock_prec_dev('ULTRACEMCO.NS')
#df_perform = df_perform.append(tmp)
for stock_id in stock_ids:
    stock_id = stock_id.upper() + '.NS'
    print(stock_id)
    tmp = stock_prec_dev(stock_id)
    tmp = tmp.reset_index()
    tmp = tmp.tail(1)
    tmp['Forecast'] = mypf[(mypf['Symbol']+'.NS') == stock_id.upper()]['Forecast'].values[0]
    df_prec_dev = df_prec_dev.append(tmp)

ASIANPAINT.NS
[*********************100%***********************]  1 of 1 completed
AXISBANK.NS
[*********************100%***********************]  1 of 1 completed
BAJAJFINSV.NS
[*********************100%***********************]  1 of 1 completed
BAJFINANCE.NS
[*********************100%***********************]  1 of 1 completed
BANKBARODA.NS
[*********************100%***********************]  1 of 1 completed
BERGEPAINT.NS
[*********************100%***********************]  1 of 1 completed
BHARTIARTL.NS
[*********************100%***********************]  1 of 1 completed
BRITANNIA.NS
[*********************100%***********************]  1 of 1 completed
CANBK.NS
[*********************100%***********************]  1 of 1 completed
CIPLA.NS
[*********************100%***********************]  1 of 1 completed
COLPAL.NS
[*********************100%***********************]  1 of 1 completed
DABUR.NS
[*********************100%***********************]  1 of 1 completed
DEEPAKNTR.NS
[************

In [8]:
stock_prec_dev('^NSEI').tail(5)

[*********************100%***********************]  1 of 1 completed


,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%
Date,,,,,,,,,
2022-12-19,18420.0,^NSEI,5.54,0.64,0.35,18812.0,15294.0,88.87,11.13
2022-12-20,18385.0,^NSEI,5.28,0.62,0.29,18812.0,15294.0,87.87,12.13
2022-12-21,18199.0,^NSEI,4.17,0.59,0.18,18812.0,15294.0,82.58,17.42
2022-12-22,18127.0,^NSEI,3.72,0.56,0.11,18812.0,15294.0,80.54,19.46
2022-12-23,17807.0,^NSEI,1.86,0.51,0.01,18812.0,15294.0,71.43,28.57


In [9]:
df_prec_dev['Upside%'] = round((df_prec_dev['Forecast']-df_prec_dev['Close Price'])*100/(df_prec_dev['Close Price']),2)
df_prec_dev = df_prec_dev.sort_values(by = 'Upside%', ascending=False)
stock_ids = df_prec_dev['Stock'].values
df_prec_dev.reset_index(drop=True, inplace=True)

In [10]:
# portfolio condition
mypf['Stock'] = mypf['Symbol'] + '.NS'
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 9


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
1,2022-12-23,440.0,ICICIPRULI.NS,-14.80,-1.36,-1.46,616.0,440.0,-0.20,100.20,640.0,45.45
2,2022-12-23,676.0,MOTILALOFS.NS,-11.40,-0.92,0.29,971.0,654.0,6.96,93.04,930.0,37.57
3,2022-12-23,1225.0,SBILIFE.NS,0.87,0.33,-0.24,1332.0,1025.0,65.02,34.98,1607.0,31.18
14,2022-12-23,3846.0,NAUKRI.NS,-8.46,-0.21,0.37,5696.0,3375.0,20.28,79.72,4677.0,21.61
17,2022-12-23,565.0,HDFCLIFE.NS,0.08,0.18,0.53,674.0,504.0,36.15,63.85,681.0,20.53
23,2022-12-23,1043.0,MUTHOOTFIN.NS,-8.23,-0.53,0.00,1540.0,956.0,14.97,85.03,1217.0,16.68
26,2022-12-23,1497.0,INFY.NS,-4.95,-0.32,-0.58,1940.0,1365.0,22.97,77.03,1715.0,14.56
27,2022-12-23,996.0,TECHM.NS,-14.16,-1.21,-0.76,1806.0,965.0,3.66,96.34,1139.0,14.36
56,2022-12-23,3498.0,DIVISLAB.NS,-6.36,-0.94,0.03,4678.0,3266.0,16.46,83.54,3560.0,1.77


In [11]:
# CCI buy
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['CCIFlag'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 15]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 3


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
7,2022-12-23,511.0,JUBLFOOD.NS,-11.85,-1.03,-1.26,786.0,464.0,14.67,85.33,643.0,25.83
16,2022-12-23,1101.0,HAVELLS.NS,-11.65,-0.80,-1.16,1406.0,1070.0,9.17,90.83,1328.0,20.62
20,2022-12-23,2483.0,TITAN.NS,-0.16,0.16,-0.73,2770.0,1936.0,65.59,34.41,2967.0,19.49


In [12]:
tmp_df = df_prec_dev[df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df['Upside%']*100/tmp_df['Upside%'].sum()

1     22.311129
2     18.442884
3     15.306072
14    10.608218
17    10.078052
23     8.188111
26     7.147415
27     7.049237
56     0.868882
Name: Upside%, dtype: float64

In [13]:
# heavy upside
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Upside%'] > 20]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 13


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-23,425.0,GODREJIND.NS,-10.05,-0.21,0.00,644.0,405.0,8.18,91.82,638.0,50.12
4,2022-12-23,378.0,TATAMOTORS.NS,-12.62,-0.69,-0.95,525.0,372.0,4.15,95.85,495.0,30.95
5,2022-12-23,235.0,ZEEL.NS,-10.63,-0.38,-1.16,332.0,209.0,21.18,78.82,303.0,28.94
6,2022-12-23,74.0,IBREALEST.NS,-17.56,-1.11,0.00,174.0,60.0,12.46,87.54,95.0,28.38
7,2022-12-23,511.0,JUBLFOOD.NS,-11.85,-1.03,-1.26,786.0,464.0,14.67,85.33,643.0,25.83
8,2022-12-23,6375.0,BAJFINANCE.NS,-6.45,-0.38,-0.85,7853.0,5284.0,42.45,57.55,8015.0,25.73
9,2022-12-23,72.0,NATIONALUM.NS,-13.43,-1.20,0.00,130.0,67.0,7.70,92.30,90.0,25.00
10,2022-12-23,3114.0,EICHERMOT.NS,-1.34,0.44,-0.97,3850.0,2256.0,53.80,46.20,3884.0,24.73
11,2022-12-23,122.0,FEDERALBNK.NS,7.41,1.75,0.00,138.0,81.0,72.72,27.28,152.0,24.59
12,2022-12-23,879.0,ICICIBANK.NS,4.76,0.83,-0.33,953.0,654.0,75.22,24.78,1094.0,24.46


In [14]:
# near 52W low
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Discount%'] > 90]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 7


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-23,425.0,GODREJIND.NS,-10.05,-0.21,0.00,644.0,405.0,8.18,91.82,638.0,50.12
4,2022-12-23,378.0,TATAMOTORS.NS,-12.62,-0.69,-0.95,525.0,372.0,4.15,95.85,495.0,30.95
9,2022-12-23,72.0,NATIONALUM.NS,-13.43,-1.20,0.00,130.0,67.0,7.70,92.30,90.0,25.00
16,2022-12-23,1101.0,HAVELLS.NS,-11.65,-0.80,-1.16,1406.0,1070.0,9.17,90.83,1328.0,20.62
21,2022-12-23,197.0,TATAPOWER.NS,-12.88,-0.44,-1.36,290.0,195.0,2.00,98.00,232.0,17.77
32,2022-12-23,576.0,BERGEPAINT.NS,-9.79,-0.78,-0.82,791.0,561.0,6.33,93.67,651.0,13.02
35,2022-12-23,378.0,WIPRO.NS,-16.63,-1.55,-0.76,722.0,375.0,0.76,99.24,425.0,12.43


In [15]:
# near 52W high
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[tmp_df['Premium%'] > 90]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Dev%_200', ascending=True)

qualified stocks: 0


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%


In [16]:
# turning around (exhaustive)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > -.5) & (tmp_df['Slope%_200'] < 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 10


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-23,425.0,GODREJIND.NS,-10.05,-0.21,0.00,644.0,405.0,8.18,91.82,638.0,50.12
53,2022-12-23,3228.0,TCS.NS,-3.02,-0.18,-0.24,4019.0,2982.0,23.76,76.24,3385.0,4.86
47,2022-12-23,1031.0,HCLTECH.NS,-1.85,-0.10,-0.29,1352.0,883.0,31.47,68.53,1117.0,8.34
43,2022-12-23,3058.0,ASIANPAINT.NS,-3.48,-0.19,-0.51,3576.0,2580.0,47.98,52.02,3361.0,9.91
39,2022-12-23,3876.0,DMART.NS,-6.54,-0.27,-0.54,4731.0,3231.0,42.97,57.03,4282.0,10.47
8,2022-12-23,6375.0,BAJFINANCE.NS,-6.45,-0.38,-0.85,7853.0,5284.0,42.45,57.55,8015.0,25.73
25,2022-12-23,8142.0,MARUTI.NS,-4.18,-0.01,-0.97,9528.0,6769.0,49.75,50.25,9356.0,14.91
5,2022-12-23,235.0,ZEEL.NS,-10.63,-0.38,-1.16,332.0,209.0,21.18,78.82,303.0,28.94
19,2022-12-23,1888.0,DEEPAKNTR.NS,-10.93,-0.09,-1.26,2660.0,1720.0,17.90,82.10,2260.0,19.70
21,2022-12-23,197.0,TATAPOWER.NS,-12.88,-0.44,-1.36,290.0,195.0,2.00,98.00,232.0,17.77


In [17]:
# turning around (compact)
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > 0)]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Upside%', ascending=False)

qualified stocks: 0


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%


In [18]:
# bull run
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] > 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 1


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
29,2022-12-23,1914.0,INDIGO.NS,2.04,0.75,1.05,2265.0,1549.0,51.01,48.99,2176.0,13.69


In [19]:
# about to turn around
tmp_df = df_prec_dev[~df_prec_dev['Stock'].isin(mypf[mypf['InPortfolio'] == 1].Stock.values)]
tmp_df = tmp_df[(tmp_df['Slope%_200'] < 0) & (tmp_df['Slope%_50'] > tmp_df['Slope%_200'])]
print('qualified stocks: '+str(tmp_df.shape[0]))
tmp_df.sort_values(by = 'Slope%_50', ascending=False)

qualified stocks: 4


,Date,Close Price,Stock,Dev%_200,Slope%_200,Slope%_50,Max,Min,Premium%,Discount%,Forecast,Upside%
0,2022-12-23,425.0,GODREJIND.NS,-10.05,-0.21,0.00,644.0,405.0,8.18,91.82,638.0,50.12
6,2022-12-23,74.0,IBREALEST.NS,-17.56,-1.11,0.00,174.0,60.0,12.46,87.54,95.0,28.38
9,2022-12-23,72.0,NATIONALUM.NS,-13.43,-1.20,0.00,130.0,67.0,7.70,92.30,90.0,25.00
35,2022-12-23,378.0,WIPRO.NS,-16.63,-1.55,-0.76,722.0,375.0,0.76,99.24,425.0,12.43
